In [37]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import exp, array, random, dot
from sklearn.model_selection import train_test_split
from math import ceil
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

np.random.seed(1)

In [38]:
# Download stopwrod text dataset
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

## Load data

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.


In [24]:
train = pd.read_csv("../data/word2vec-nlp/labeledTrainData.tsv", delimiter="\t")
test = pd.read_csv("../data/word2vec-nlp/testData.tsv", delimiter="\t")
print(f"Size of train dataset: {train.shape}")
print(f"Size of test dataset: {test.shape}")

Size of train dataset: (25000, 3)
Size of test dataset: (25000, 2)


In [25]:
train.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [27]:
train["review"][0]

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

## Preprocess data

Data preprocessing includes the following steps:
* Remove HTML tags
* Remove non-letter symbols        
* Convert to lower case, split into individual words
* Remove stop words with the help of nltk library


In [7]:
def review_to_words(raw_review: str) -> str:
    """Converts a raw review to a processed string of words."""

    review_text = BeautifulSoup(raw_review).get_text() 
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    words = letters_only.lower().split()                             
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops]   
    return(" ".join( meaningful_words ))   

In [8]:
clean_review = review_to_words(train["review"][0])
print(clean_review)

stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate working

In [30]:
num_reviews = train["review"].size
clean_train_reviews = []

for i in range(0, num_reviews):
    if((i + 1) % 5000 == 0):
        print(f"Review {i+1} of {num_reviews}")
    clean_train_reviews.append(review_to_words(train["review"][i]))
print("All reviews have been cleaned out.")

Review 5000 of 25000
Review 10000 of 25000
Review 15000 of 25000
Review 20000 of 25000
Review 25000 of 25000
All reviews have been cleaned out.


In [35]:
# Create bag of words. The Bag of Words model learns a vocabulary from all of the documents, 
# then models each document by counting the number of times each word appears.
# We are going to use 5000 most frequent words to create a vocab
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 
train_data_features = vectorizer.fit_transform(clean_train_reviews)
train_data_features = train_data_features.toarray()
print("Size of featurized train data:", train_data_features.shape)


Size of featurized train data: (25000, 5000)


In [36]:
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names_out()
print(vocab)

['abandoned' 'abc' 'abilities' ... 'zombie' 'zombies' 'zone']


## Build a model


Let's use Random Forest classifier with 100 trees:

In [39]:
forest = RandomForestClassifier(n_estimators = 100) 
forest = forest.fit(train_data_features, train["sentiment"])

## Submit the prediction

In [40]:
clean_test_reviews = [] 
num_reviews = len(test["review"])

print("Cleaning and parsing the test set movie reviews...")
for i in range(0, num_reviews):
    if((i + 1) % 5000 == 0):
        print(f"Review {i+1} of {num_reviews}")
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)

submission = pd.DataFrame(data={"id": test["id"], "sentiment": result})
submission.to_csv("../data/word2vec-nlp/BagOfWordsSubmission.csv", index=False, quoting=3)
submission.head()

Cleaning and parsing the test set movie reviews...


D:\projects\venvs\venv310_basic\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


Review 5000 of 25000
Review 10000 of 25000
Review 15000 of 25000
Review 20000 of 25000
Review 25000 of 25000


,id,sentiment
0,12311_10,1
1,8348_2,0
2,5828_4,1
3,7186_2,1
4,12128_7,1
